# 基于深度学习的 MZM 导频法闭环控制（数据集生成 / 训练 / 推理）

本 Notebook 的目标：
- 只使用 PD 输出的导频 1f/2f **功率(dBm)**（不使用 DC 光功率/不依赖高速 RF 可观测）。
- 目标角度范围：**0–180°**（对应单个周期半波）。
- 将 **数据集生成 → 训练 → 推理/回放** 拆开，便于复用先前产物。

In [ ]:
from __future__ import annotations

import importlib
import numpy as np
import matplotlib.pyplot as plt

# 高分辨率内联图像
%config InlineBackend.figure_format = 'retina'

# Reusable pipeline module (reload to pick up edits)
import mzm.dither_controller as dc
dc = importlib.reload(dc)

DATASET_PATH = 'artifacts/dither_dataset_dbm_hist.npz'
MODEL_PATH = 'artifacts/dither_policy_dbm_hist.pt'

BEST_MODEL_PATH = 'artifacts/dither_policy_dbm_hist_best.pt'
BEST_EVAL_PATH = 'artifacts/dither_policy_dbm_hist_best_eval.npz'

In [ ]:
import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")

if torch.cuda.is_available():
    print(f"✅ CUDA is available. Device: {torch.cuda.get_device_name(0)}")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("✅ MPS (Metal Performance Shaders) is available on macOS.")
else:
    print("ℹ️ No GPU acceleration detected. Using CPU.")

## 数据集生成

数据集会保存到 `artifacts/`，下次可直接加载复用。

In [ ]:
device_params = dc.DeviceParams()

# 导频参数配置
dither_params = dc.DitherParams(
    V_dither_amp=0.05,    # 导频幅度 (V)
    f_dither=1e3,        # 导频频率 (Hz), 默认 1 kHz
    Fs=2e5,               # 采样率 (Hz), 默认 200 kHz
    n_periods=120,        # 采集周期数
)

# RF 训练开关
ENABLE_RF_TRAINING = True  # 开关：是否在训练数据中加入 RF 信号

# RF 信号参数
RF_AMP_FOR_TRAINING = 0.2   # RF 信号幅度 (V)，建议值：0.1 ~ 0.3
RF_FREQ = 1e9               # RF 信号频率 (Hz)，默认 1 GHz

# Feature mode
FEATURE_MODE = 'theta_est_hist'  # 'dc_norm_hist' | 'shape_norm_bessel' | 'theta_est_hist'

# 打印当前导频配置
print(f"Dither params: f={dither_params.f_dither/1e3:.1f} kHz, "
      f"Amp={dither_params.V_dither_amp:.3f} V, "
      f"Fs={dither_params.Fs/1e6:.1f} MHz, "
      f"periods={dither_params.n_periods}")
print('feature_mode:', FEATURE_MODE)

if ENABLE_RF_TRAINING:
    print(f"⚡ RF Training ENABLED: V_rf_amp={RF_AMP_FOR_TRAINING} V, f_rf={RF_FREQ/1e9:.1f} GHz")
else:
    print(f"📴 RF Training DISABLED (no RF signal during training)")

# Generate (or overwrite) dataset artifact
# 增大样本量以覆盖 0 度附近的物理死区（灵敏度低区域）
ds0 = dc.generate_dataset_dbm_hist(
    device_params=device_params,
    dither_params=dither_params,
    n_samples=60000,  # Increased from 20000 to 60000 to improve edge/null-point accuracy
    seed=None,
    teacher_gain=0.5,
    max_step_V=0.2,
    feature_mode=FEATURE_MODE,
    accel='auto',
    torch_batch=128,
    V_rf_amp=RF_AMP_FOR_TRAINING if ENABLE_RF_TRAINING else 0.0,
    f_rf=RF_FREQ,
)
dc.save_dataset(ds0, DATASET_PATH)
print('saved dataset:', DATASET_PATH)

# If you used CUDA/MPS, PyTorch may keep a caching allocator reserve in VRAM.
# This is normal (not a leak). If you want to release it back to the OS:
import gc
import torch
del ds0
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    torch.mps.empty_cache()

ds = dc.load_dataset(DATASET_PATH)
print('Xn:', ds['Xn'].shape, 'y:', ds['y'].shape)
print('mu:', ds['mu'].shape, 'sigma:', ds['sigma'].shape)
print('device_params:', ds['device_params'])
print('dither_params:', ds['dither_params'])
print(f"Training RF config: V_rf_amp={ds['V_rf_amp']:.3f} V, f_rf={ds['f_rf']/1e9:.1f} GHz")
print('feature_version:', ds.get('feature_version', 'legacy'))

## 训练

训练输出保存到 `artifacts/dither_policy_dbm_hist.pt`，可重复加载复用。

In [ ]:
import importlib
import mzm.dither_controller as dc
dc = importlib.reload(dc)

ds = dc.load_dataset(DATASET_PATH)

# Note: accel='cpu' is often faster for small models due to lower latency
# We have optimized the training loop to remove DataLoader overhead.
# 调整训练参数以避免过拟合噪声，并提高在死区附近的泛化能力
model = dc.train_policy(
    Xn=ds['Xn'],
    y=ds['y'],
    epochs=2000,   # 降低轮数，防止对 0 度附近的噪声过拟合 (was 10000)
    batch=1024,    # 增大 Batch Size，使梯度更新更平滑 (was 256)
    lr=5e-4,       # 降低学习率，精细搜索最优解 (was 1e-3)
    hidden=128,    # 增加模型容量，以拟合非线性极值点特征 (was 64)
    depth=3,
    seed=None,
    accel='auto',
 )

dc.save_model(
    model=model,
    mu=ds['mu'],
    sigma=ds['sigma'],
    device_params=ds['device_params'],
    dither_params=ds['dither_params'],
    feature_version=ds.get('feature_version', 'legacy'),
    path=MODEL_PATH,
 )
print('saved model:', MODEL_PATH)

## 推理/回放（单目标角度 + 逐轮过程 + 前后光谱对比）

- 选择一个 `target_deg` 作为锁定目标；
- 展示每一轮迭代的观测与动作（`p1/p2`、差分、`ΔV`、`V`、`θ`、误差）；
- 在初始偏置与终止偏置分别绘制一次**光谱/电谱**用于前后对比。

In [ ]:
from mzm.model import simulate_mzm, bias_to_theta_rad
from mzm.plot import plot_optical_spectrum_osa, plot_electrical_spectrum, plot_rollout_convergence

model, meta = dc.load_model(MODEL_PATH)
print('model feature_version:', meta.get('feature_version', 'legacy'))

# Choose a single target working point (deg)
target_deg = 0.0

# RF 推理开关
ENABLE_RF_INFERENCE = True  # 开关：是否在推理时加入 RF 信号

# RF 信号参数
RF_AMP_FOR_INFERENCE = 0.2   # RF 信号幅度 (V)
RF_FREQ_INFERENCE = 1e9      # RF 信号频率 (Hz)

# Initial bias: random by default.
# - Set init_seed to an integer for reproducible runs (e.g., 1).
# - Keep it as None to re-sample a different V_init each run.
init_seed = None
rng = np.random.default_rng(init_seed)
V_init = float(rng.uniform(0.0, float(meta['device_params'].Vpi_DC)))

theta_init_deg = float(np.rad2deg(bias_to_theta_rad(V_init, Vpi_DC=float(meta['device_params'].Vpi_DC))))

print('target_deg     :', target_deg)
print('init V (V)     :', V_init)
print('init theta (deg):', theta_init_deg)

if ENABLE_RF_INFERENCE:
    print(f'⚡ RF Inference ENABLED: V_rf_amp={RF_AMP_FOR_INFERENCE} V, f_rf={RF_FREQ_INFERENCE/1e9:.1f} GHz')
else:
    print(f'📴 RF Inference DISABLED (no RF signal during rollout)')

# --- (A) Pre-control spectra at initial bias ---
print('\n[pre] spectra at V_init')
sim_pre = simulate_mzm(
    V_bias=V_init,
    Vpi_RF=float(meta['device_params'].Vpi_DC),
    Vpi_DC=float(meta['device_params'].Vpi_DC),
    ER_dB=float(meta['device_params'].ER_dB),
    IL_dB=float(meta['device_params'].IL_dB),
    Pin_dBm=float(meta['device_params'].Pin_dBm),
    Responsivity=float(meta['device_params'].Responsivity),
    R_load=float(meta['device_params'].R_load),
    # RF settings used only for spectrum illustration
    f_rf=1e9,
    V_rf_amp=0.2,
    Fs=100e9,
    T_total=5e-6,
 )
print('pre  P_pd_avg_dBm:', sim_pre.P_pd_avg_dBm)
# plot_optical_spectrum_osa(sim_pre, f_rf_hz=1e9)
# plot_electrical_spectrum(sim_pre, f_rf_hz=1e9)

# --- (B) Closed-loop rollout (dBm-only + history) ---
steps = 60
r = dc.rollout_dbm_hist(
    model=model,
    mu=meta['mu'],
    sigma=meta['sigma'],
    device_params=meta['device_params'],
    dither_params=meta['dither_params'],
    theta_target_deg=float(target_deg),
    V_init=V_init,
    steps=steps,
    accel='cpu',
    V_rf_amp=RF_AMP_FOR_INFERENCE if ENABLE_RF_INFERENCE else 0.0,
    f_rf=RF_FREQ_INFERENCE,
    max_step_V=0.2,
    feature_mode='theta_est_hist',
 )

V_final = float(r['V'][-1])
final_err = float(r['err_deg'][-1])
print('\n[rollout] final V:', V_final)
print('[rollout] final wrapped err (deg):', f"{final_err:+.2f}")

# Print per-iteration trace
print('\nPer-iteration trace:')
print(' iter |    V(V) |  theta(deg) | theta_est |  err(deg) |   dV(V) |  h1_raw |  h2_raw |     x0 |     x1 |  dx0 |  dx1 |  b_est')
for k in range(steps):
    print(
        f"{k+1:5d} |"
        f" {r['V'][k]:7.4f} |"
        f" {r['theta_deg'][k]:10.3f} |"
        f" {r['theta_est_deg'][k]:9.3f} |"
        f" {r['err_deg'][k]:8.3f} |"
        f" {r['dv'][k]:7.4f} |"
        f" {r['h1_norm_raw'][k]:7.4f} |"
        f" {r['h2_norm_raw'][k]:7.4f} |"
        f" {r['h1_norm'][k]:7.4f} |"
        f" {r['h2_norm'][k]:7.4f} |"
        f" {r['dh1_norm'][k]:5.3f} |"
        f" {r['dh2_norm'][k]:5.3f} |"
        f" {r['b_est'][k]:6.3f}"
     )

# Plot convergence curve
rf_status = f" (RF: {RF_AMP_FOR_INFERENCE*1000:.0f}mV)" if ENABLE_RF_INFERENCE else " (no RF)"
plot_rollout_convergence(r['err_deg'], target_deg=target_deg, rf_info=rf_status, title_prefix='Closed-loop convergence (dBm-only + history)')
plt.show()

# --- (C) Post-control spectra at final bias ---
print('\n[post] spectra at V_final')
sim_post = simulate_mzm(
    V_bias=V_final,
    Vpi_RF=float(meta['device_params'].Vpi_DC),
    Vpi_DC=float(meta['device_params'].Vpi_DC),
    ER_dB=float(meta['device_params'].ER_dB),
    IL_dB=float(meta['device_params'].IL_dB),
    Pin_dBm=float(meta['device_params'].Pin_dBm),
    Responsivity=float(meta['device_params'].Responsivity),
    R_load=float(meta['device_params'].R_load),
    f_rf=1e9,
    V_rf_amp=0.2,
    Fs=100e9,
    T_total=5e-6,
 )
print('post P_pd_avg_dBm:', sim_post.P_pd_avg_dBm)
plot_optical_spectrum_osa(sim_post, f_rf_hz=1e9)
plot_electrical_spectrum(sim_post, f_rf_hz=1e9)

## 模型评估与对比

扫描 0-180° 目标角度，每个角度重复推理 50 次（根据显存大小调整推理次数）。
- **可视化**：绘制平均误差曲线，并使用阴影区域表示方差（标准差范围），类似 K 线图的概念展示波动情况。
- **缓存**：自动加载历史最佳模型的评估结果（`artifacts/dither_policy_dbm_hist_best_eval.npz`），避免重复计算。

In [ ]:
from mzm.eval import evaluate_current_and_best
from mzm.plot import plot_target_sweep_stats

# RF 评估开关
ENABLE_RF_EVALUATION = True  # 开关：是否在评估时加入 RF 信号

# RF 信号参数
RF_AMP_FOR_EVALUATION = 0.2   # RF 信号幅度 (V)
RF_FREQ_EVALUATION = 1e9      # RF 信号频率 (Hz)

# Print RF evaluation config
if ENABLE_RF_EVALUATION:
    print(f"⚡ RF Evaluation ENABLED: V_rf_amp={RF_AMP_FOR_EVALUATION} V, f_rf={RF_FREQ_EVALUATION/1e9:.1f} GHz")
else:
    print(f"📴 RF Evaluation DISABLED (no RF signal during evaluation)")

# Get RF parameters for evaluation
eval_V_rf_amp = RF_AMP_FOR_EVALUATION if ENABLE_RF_EVALUATION else 0.0
eval_f_rf = RF_FREQ_EVALUATION
rf_info_str = f" (RF: {eval_V_rf_amp*1000:.0f}mV)" if ENABLE_RF_EVALUATION else " (no RF)"

# Evaluate current & best
current_stats, best_stats, update_action = evaluate_current_and_best(
    model_path=MODEL_PATH,
    best_model_path=BEST_MODEL_PATH,
    best_eval_path=BEST_EVAL_PATH,
    steps=60,
    n_repeats=50,
    V_rf_amp=eval_V_rf_amp,
    f_rf=eval_f_rf,
    accel='auto',
    seed=None,
)

# Plot
plot_target_sweep_stats(current_stats, best_stats, rf_info=rf_info_str)
plt.show()

if current_stats:
    print(f"Current Model Overall Mean MAE: {np.mean(current_stats['mean']):.4f} deg")
if best_stats:
    print(f"Best Model Overall Mean MAE: {np.mean(best_stats['mean']):.4f} deg")
if update_action:
    print(update_action)

## RF 功率变化鲁棒性测试

评估模型在不同 **RF 调制信号功率**（1GHz 高频信号的 `V_rf_amp`）下的性能表现。

- **背景**：模型训练时没有 RF 信号干扰（`V_rf_amp=0`）
- **测试目的**：验证当系统存在高频 RF 数据调制时，导频控制器能否正常工作
- **扫描范围**：`V_rf_amp` 从 0V 到 0.5V（对应不同的调制深度）

In [ ]:
from mzm.eval import evaluate_rf_power_robustness
from mzm.plot import plot_rf_power_robustness_curve

# 运行评估
rf_robustness_results = evaluate_rf_power_robustness(
    model_path=MODEL_PATH,
    V_rf_amp_values=[0.0, 0.02, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5],
    f_rf=1e9,  # 1 GHz
    target_deg=90.0,
    steps=60,
    n_repeats=100,
    accel='auto',
)

plot_rf_power_robustness_curve(rf_robustness_results)
plt.show()

# 打印统计信息
if rf_robustness_results is not None:
    V_rf = rf_robustness_results['V_rf_amp_values']
    means = rf_robustness_results['mean_errors']
    stds = rf_robustness_results['std_errors']
    f_rf_ghz = rf_robustness_results['f_rf'] / 1e9
    print(f"RF 功率鲁棒性测试结果 (f_rf = {f_rf_ghz:.1f} GHz)")
    print(f"{'V_rf (mV)':<12} {'平均误差(°)':<14} {'标准差(°)':<12}")
    print("-" * 40)
    for i, v in enumerate(V_rf):
        print(f"{v*1000:<12.1f} {means[i]:<14.4f} {stds[i]:<12.4f}")
    
    baseline_error = means[0]  # V_rf = 0 时的误差
    for threshold, label in [(1.5, "50%"), (2.0, "100%")]:
        robust_mask = means < baseline_error * threshold
        if np.any(robust_mask):
            max_robust_V = V_rf[robust_mask].max()
            print(f"\n鲁棒性阈值 (误差增加 < {label}): V_rf < {max_robust_V*1000:.1f} mV")

In [ ]:
from mzm.eval import evaluate_rf_robustness_multi_target
from mzm.plot import plot_rf_robustness_heatmap

# 运行多角度评估
multi_target_results = evaluate_rf_robustness_multi_target(
    model_path=MODEL_PATH,
    V_rf_amp_values=[0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5],
    f_rf=1e9,
    target_angles=[0, 30, 60, 90, 120, 150, 180],
    steps=60,
    n_repeats=30,
    accel='auto',
)

plot_rf_robustness_heatmap(multi_target_results)
plt.show()

# 打印总结
if multi_target_results is not None:
    V_rf = multi_target_results['V_rf_amp_values']
    angles = multi_target_results['target_angles']
    means = multi_target_results['mean_errors']
    f_rf_ghz = multi_target_results['f_rf'] / 1e9
    print(f"多角度 RF 功率鲁棒性总结 (f_rf = {f_rf_ghz:.1f} GHz)")
    print(f"基线 (V_rf=0.2) 平均误差: {np.mean(means[4, :]):.4f}°")
    worst_idx = np.unravel_index(np.argmax(means), means.shape)
    print(f"最差条件误差: {np.max(means):.4f}° (V_rf={V_rf[worst_idx[0]]*1000:.0f}mV, angle={angles[worst_idx[1]]}°)")
    print(f"整体平均误差: {np.mean(means):.4f}°")

## 光功率变化鲁棒性测试

评估模型在不同 **输入光功率**（`Pin_dBm`）下的性能表现。

- **背景**：模型训练时使用固定光功率（`Pin_dBm=10 dBm`）
- **测试目的**：验证当输入光功率变化时（如激光器老化、光纤振动），导频控制器能否正常工作
- **扫描范围**：`Pin_dBm` 从 0 dBm 到 15 dBm
- **DC 归一化优势**：使用 `h1/pd_dc` 和 `h2/pd_dc` 作为特征可以抵消光功率变化的影响

In [ ]:
from mzm.eval import evaluate_optical_power_robustness
from mzm.plot import plot_optical_power_robustness_curve

# 运行评估
optical_power_results = evaluate_optical_power_robustness(
    model_path=MODEL_PATH,
    Pin_dBm_values=[0.0, 3.0, 5.0, 7.0, 10.0, 12.0, 15.0],
    target_deg=90.0,
    steps=60,
    n_repeats=100,
    V_rf_amp=0.2,  # 同时加入 RF 信号
    f_rf=1e9,
    accel='auto',
)

plot_optical_power_robustness_curve(optical_power_results)
plt.show()

# 打印统计信息
if optical_power_results is not None:
    Pin = optical_power_results['Pin_dBm_values']
    means = optical_power_results['mean_errors']
    stds = optical_power_results['std_errors']
    training_Pin = optical_power_results['training_Pin_dBm']
    print(f"光功率鲁棒性测试结果")
    print(f"{'Pin (dBm)':<12} {'平均误差(°)':<14} {'标准差(°)':<12}")
    print("-" * 40)
    for i, p in enumerate(Pin):
        marker = " <-- Training" if abs(p - training_Pin) < 0.1 else ""
        print(f"{p:<12.1f} {means[i]:<14.4f} {stds[i]:<12.4f}{marker}")
    
    baseline_idx = np.argmin(np.abs(Pin - training_Pin))
    baseline_error = means[baseline_idx]
    print(f"\n训练点 ({training_Pin:.0f} dBm) 误差: {baseline_error:.4f}°")
    
    if len(Pin) > 1:
        higher_power_idx = Pin > training_Pin
        if np.any(higher_power_idx):
            max_higher = means[higher_power_idx].max()
            max_higher_pin = Pin[higher_power_idx][np.argmax(means[higher_power_idx])]
            print(f"最高功率 ({max_higher_pin:.0f} dBm) 误差: {max_higher:.4f}°")
        
        lower_power_idx = Pin < training_Pin
        if np.any(lower_power_idx):
            max_lower = means[lower_power_idx].max()
            max_lower_pin = Pin[lower_power_idx][np.argmax(means[lower_power_idx])]
            print(f"最低功率 ({max_lower_pin:.0f} dBm) 误差: {max_lower:.4f}°")

# 偏置漂移仿真 (Bias Drift Simulation)

使用训练好的模型，测试其在连续偏置漂移（Ramp Disturbance）下的鲁棒性。
这可以评估控制器是否能够“抵抗”环境变化，保持工作点稳定。

In [ ]:
import mzm.drift_simulation as drift_sim

# 加载模型
model_path = MODEL_PATH
model, meta = dc.load_model(model_path)

mu = meta['mu']
sigma = meta['sigma']
device_params = meta['device_params']
dither_params = meta['dither_params']

print(f"Loaded model from {model_path}")
print(f"Dither Params: {dither_params}")

# 运行仿真
# 可以尝试调整 drift_step_rate (例如 0.001 vs 0.002) 观察控制器的极限
hist = drift_sim.simulate_control_loop_with_drift(
    model, mu, sigma, device_params, dither_params, 
    n_steps=10000, 
    target_theta_deg=90.0,
    drift_step_rate=0.0005
)

# 绘图
drift_sim.plot_results(hist, target_deg=90.0, show=True)

# --- Vpi 漂移鲁棒性仿真 ---
# vpi_abs_step_rate 是“每 step 的 Vpi 真实值漂移量 (V/step)”，例如 1e-6 表示 10000 steps 漂移 0.01V
hist_vpi = drift_sim.simulate_control_loop_with_vpi_drift(
    model, mu, sigma, device_params, dither_params,
    n_steps=10000,
    target_theta_deg=90.0,
    vpi_abs_step_rate=1e-5,  # 5.0V -> 5.1V
    controller_knows_vpi=False,
)

drift_sim.plot_results_vpi_drift(hist_vpi, target_deg=90.0, show=True, vpi_y='true')